# Анализ лояльности пользователей Яндекс Афиши

## Цели и задачи проекта

**Цель:** Команда маркетинга хочет лучше понимать поведение пользователей. Для этого они просят вас провести исследовательский анализ данных, чтобы понять, какие пользователи с большей вероятностью возвращаются на платформу и делают заказы повторно. Это позволит:
- Быстро выявлять перспективных клиентов и предлагать им персонализированные условия.
- Точно настраивать рекламу на аудитории с высокой вероятностью возврата.
- Оптимизировать маркетинговые бюджеты.
- Повысить общий уровень удержания клиентов.

**Задачи:**
1. **Создание профиля пользователя**
    - Построить профиль пользователя — для каждого пользователя
    - Оценить данные: достаточно ли их по объёму, есть ли аномальные значения в данных о количестве заказов и среднем количестве билетов?
    
    
2. **Исследовательский анализ данных**
    - Исследовать признаки, описывающие первый заказ пользователя, и выяснить, влияют ли они на вероятность возвращения пользователя.  
    
    - Опираясь на выводы из задач выше, проверить продуктовые гипотезы:
        - Гипотеза 1. Тип мероприятия влияет на вероятность возврата на Яндекс Афишу: пользователи, которые совершили первый заказ на спортивные мероприятия, совершают повторный заказ чаще, чем пользователи, оформившие свой первый заказ на концерты.
        - Гипотеза 2. В регионах, где больше всего пользователей посещают мероприятия, выше доля повторных заказов, чем в менее активных регионах.
        
    - Изучить количественные характеристики заказов пользователей, чтобы узнать среднюю выручку сервиса с заказа и количество билетов, которое пользователи обычно покупают.
    
    - Изучите временные параметры, связанные с первым заказом пользователей:
        - день недели первой покупки;
        - время с момента первой покупки — лайфтайм;
        - средний интервал между покупками пользователей с повторными заказами.


3. **Корреляционный анализ количества покупок и признаков пользователя**
    - Изучить, какие характеристики первого заказа и профиля пользователя могут быть связаны с числом покупок. Использовать универсальный коэффициент корреляции phi_k, который позволяет анализировать как числовые, так и категориальные признаки.

## Описание данных

`user_id` — уникальный идентификатор пользователя, совершившего заказ;  

`device_type_canonical` — тип устройства, с которого был оформлен заказ ( mobile — мобильные устройства, desktop — стационарные);  

`order_id` — уникальный идентификатор заказа;  

`order_dt` — дата создания заказа (используйте данные created_dt_msk );  

`order_ts` — дата и время создания заказа (используйте данные created_ts_msk );  

`currency_code` — валюта оплаты;  

`revenue` — выручка от заказа;  

`tickets_count` — количество купленных билетов;  

`days_since_prev` — количество дней от предыдущей покупки пользователя, для пользователей с одной покупкой — значение пропущено;

`event_id` — уникальный идентификатор мероприятия;  

`service_name` — название билетного оператора;  

`event_type_main` — основной тип мероприятия (театральная постановка, концерт и так далее);  

`region_name` — название региона, в котором прошло мероприятие;  

`city_name` — название города, в котором прошло мероприятие.
___

## Загрузка данных

Импортируем необходимые библиотеки, выгрузим данные.

In [8]:
!pip install psycopg2

In [9]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

In [13]:
#создадим словарь с необходимыми для подключения к БД данными
db_config = {'user': 'praktikum_student',
             'pwd': 'Sdf4$2;d-d30pp',
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,
             'db': 'data-analyst-afisha'
             }

In [14]:
#создадим строку для авторизации и передадим ее в функцию create_engine
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db'],
)
engine = create_engine(connection_string)

In [15]:
#запишем запрос к БД в переменную
query = """
SELECT user_id,
  device_type_canonical,
  order_id,
  created_dt_msk AS order_dt,
  created_ts_msk AS order_ts,
  currency_code,
  revenue,
  tickets_count,
  created_dt_msk::date - LAG(created_dt_msk) OVER(PARTITION BY user_id ORDER BY created_dt_msk)::date AS days_since_prev,
  event_id,
  e.event_name_code AS event_name,
  e.event_type_main,
  service_name,
  r.region_name,
  c.city_name
FROM afisha.purchases p 
INNER JOIN afisha.events e USING(event_id)
LEFT JOIN afisha.city c USING(city_id)
LEFT JOIN afisha.regions r USING(region_id)
WHERE device_type_canonical IN ('mobile','desktop') AND e.event_type_main != 'фильм'
ORDER BY user_id
"""

In [16]:
#создадим датафрейм из таблицы
df = pd.read_sql_query(query,con=engine)